# ChEMBL webresource client examples

The library helps accessing ChEMBL data and cheminformatics tools from Python. You don't need to know how to write SQL. You don't need to know how to interact with REST APIs. You don't need to compile or install any cheminformatics frameworks. Results are cached.

The client handles interaction with the HTTPS protocol and caches all results in the local file system for faster retrieval. Abstracting away all network-related tasks, the client provides the end user with a convenient interface, giving the impression of working with a local resource. Design is based on the Django QuerySet interface. The client also implements lazy evaluation of results, which means it will only evaluate a request for data when a value is required. This approach reduces number of network requests and increases performance.

## Available data entities

You can list available data entities using the following code

In [2]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_class', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


## Available filters

The design of the client is based on Django QuerySet (https://docs.djangoproject.com/en/1.11/ref/models/querysets) and most important lookup types are supported. These are:

- exact
- iexact
- contains
- icontains
- in
- gt
- gte
- lt
- lte
- startswith
- istartswith
- endswith
- iendswith
- range
- isnull
- regex
- iregex

## Only operator

`only` is a special method allowing to limit the results to a selected set of fields. only should take a single argument: a list of fields that should be included in result. Specified fields have to exists in the endpoint against which only is executed. Using only will usually make an API call faster because less information returned will save bandwidth. The API logic will also check if any SQL joins are necessary to return the specified field and exclude unnecessary joins with critically improves performance.

Please note that only has one limitation: a list of fields will ignore nested fields i.e. calling only(['molecule_properties__alogp']) is equivalent to only(['molecule_properties']).

For many 2 many relationships only will not make any SQL join optimisation.

# Molecules

Molecule records may be retrieved in a number of ways, such as lookup of single molecules using various identifiers or searching for compounds via similarity.

## Find a molecule by pref_name


In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
mols = molecule.filter(pref_name__iexact='aspirin')
display(mols)
pd.DataFrame(mols)

## Find a molecule by its synonyms

- in case it is not found by pref_name
- Use the `only` method where you can specify fields you want to be included in response

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
mols = molecule.filter(molecule_synonyms__molecule_synonym__iexact='viagra').only('molecule_chembl_id')
display(mols)
pd.DataFrame(mols)

## Get a single molecule by ChEMBL id

All the main entities in the ChEMBL database have a ChEMBL ID. It is a stable identifier designed for straightforward lookup of data.

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
m1 = molecule.filter(chembl_id='CHEMBL192').only(['molecule_chembl_id', 'pref_name', 'molecule_structures'])
display(m1)
pd.DataFrame(m1)

## Get many molecules by id

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
mols = molecule.filter(molecule_chembl_id__in=['CHEMBL25', 'CHEMBL192', 'CHEMBL27']).only(['molecule_chembl_id', 'pref_name'])
display(mols)
pd.DataFrame(mols)

## Display a molecule image

In [ ]:
from chembl_webresource_client.new_client import new_client
from IPython.display import SVG

image = new_client.image
image.set_format('svg')
SVG(image.get('CHEMBL25'))

## Get a single molecule by standard inchi key

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
mol = molecule.filter(molecule_structures__standard_inchi_key='BSYNRYMUTXBXSQ-UHFFFAOYSA-N').only(['molecule_chembl_id', 'pref_name', 'molecule_structures'])
display(mol)
pd.DataFrame(mol)

## Find compounds similar to given SMILES query with similarity threshold of 70%

In [ ]:
from chembl_webresource_client.new_client import new_client

similarity = new_client.similarity
res = similarity.filter(smiles="CO[C@@H](CCC#C\C=C/CCCC(C)CCCCC=C)C(=O)[O-]", similarity=70).only(['molecule_chembl_id', 'similarity'])
for i in res:
    print(i)
    
pd.DataFrame(res)

## Find compounds similar to aspirin (CHEMBL25) with similarity threshold of 70%


In [ ]:
from chembl_webresource_client.new_client import new_client

similarity = new_client.similarity
res = similarity.filter(chembl_id='CHEMBL25', similarity=70).only(['molecule_chembl_id', 'pref_name', 'similarity'])
display(res)
pd.DataFrame(res)

## Find compounds with the same connectivity

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
res = molecule.filter(molecule_structures__canonical_smiles__connectivity='CN(C)C(=N)N=C(N)N').only(['molecule_chembl_id', 'pref_name'])
for i in res:
    print(i)
pd.DataFrame(res)

## Get all approved drugs

using `order_by` to sort them by molecular weight

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
approved_drugs = molecule.filter(max_phase=4).order_by('molecule_properties__mw_freebase')
approved_drugs
pd.DataFrame(approved_drugs)

## Get approved drugs for lung cancer

In [ ]:
from chembl_webresource_client.new_client import new_client

drug_indication = new_client.drug_indication
molecules = new_client.molecule

lung_cancer_ind = drug_indication.filter(efo_term__icontains="LUNG CARCINOMA")
lung_cancer_mols = molecules.filter(
    molecule_chembl_id__in=[x['molecule_chembl_id'] for x in lung_cancer_ind])

print(len(lung_cancer_mols))
pd.DataFrame(lung_cancer_ind)

## Filter drugs by approval year and name

In [ ]:
from chembl_webresource_client.new_client import new_client

drug = new_client.drug
res = drug.filter(first_approval__gte=1980).filter(usan_stem="-azosin")
pd.DataFrame(res)

## Get all biotherapeutic molecules

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
biotherapeutics = molecule.filter(biotherapeutic__isnull=False)
print(len(biotherapeutics))
d = pd.DataFrame(biotherapeutics)
d.head(1)

## Get molecules with molecular weight <= 300

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
light_molecules = molecule.filter(molecule_properties__mw_freebase__lte=300)

print(len(light_molecules))
print(light_molecules)
#d = pd.DataFrame(light_molecules)
#d.head(1)

## Get molecules with molecular weight <= 300 AND pref_name ending with nib

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
light_nib_molecules = molecule.filter(molecule_properties__mw_freebase__lte=300, pref_name__iendswith="nib").only(['molecule_chembl_id', 'pref_name'])

d = pd.DataFrame(light_nib_molecules)
d

## Get all molecules in ChEMBL with no Rule-of-Five violations

In [ ]:
from chembl_webresource_client.new_client import new_client

molecule = new_client.molecule
no_violations = molecule.filter(molecule_properties__num_ro5_violations=0)

print(len(no_violations))
print(no_violations)

# Activities

## Get all IC50 activities related to the hERG target

In [ ]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
activity = new_client.activity
herg = target.filter(pref_name__iexact='hERG').only('target_chembl_id')[0]
herg_activities = activity.filter(target_chembl_id=herg['target_chembl_id']).filter(standard_type="IC50")

print(len(herg_activities))
print(herg_activities)

## Get all activities for a specific target with assay type B (binding):

In [ ]:
from chembl_webresource_client.new_client import new_client

activity = new_client.activity
res = activity.filter(target_chembl_id='CHEMBL3938', assay_type='B')

print(len(res))
print(res)

## Get all activities with a pChEMBL value for a molecule

In [ ]:
from chembl_webresource_client.new_client import new_client

activities = new_client.activity
res = activities.filter(molecule_chembl_id="CHEMBL25", pchembl_value__isnull=False)

print(len(res))
print(res)

In [ ]:
res[0]

## Search for ADMET-related inhibitor assays (type A)

In [ ]:
from chembl_webresource_client.new_client import new_client
assay = new_client.assay
res = assay.filter(description__icontains='inhibit', assay_type='A')

print(len(res))
print(res)

# Tissues

## Get tissue by BTO ID

In [ ]:
from chembl_webresource_client.new_client import new_client

tissue = new_client.tissue
res = tissue.filter(bto_id="BTO:0001073")
print(len(res))
pd.DataFrame(res)

## Get tissue by Caloha id

In [ ]:
from chembl_webresource_client.new_client import new_client

tissue = new_client.tissue
res = tissue.filter(caloha_id="TS-0490")
print(len(res))
pd.DataFrame(res)

## Get tissue by Uberon id

In [ ]:
from chembl_webresource_client.new_client import new_client

tissue = new_client.tissue
res = tissue.filter(uberon_id="UBERON:0000173")
print(len(res))
pd.DataFrame(res)

## Get tissue by name

In [ ]:
from chembl_webresource_client.new_client import new_client

tissue = new_client.tissue
res = tissue.filter(pref_name__istartswith='blood')
print(len(res))
pd.DataFrame(res)

# Cells

## Get cell line by cellosaurus id

In [ ]:
from chembl_webresource_client.new_client import new_client

cell_line = new_client.cell_line
res = cell_line.filter(cellosaurus_id="CVCL_0417")
print(len(res))
pd.DataFrame(res)

# Targets

## Find a target by gene name

In [ ]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
gene_name = 'BRD4'
res = target.filter(target_synonym__icontains=gene_name).only(['organism', 'pref_name', 'target_type'])
for i in res:
    display(i)
    
pd.DataFrame(res)

# Utils

## Convert SMILES to CTAB

In [ ]:
from chembl_webresource_client.utils import utils

aspirin = utils.smiles2ctab('O=C(Oc1ccccc1C(=O)O)C')

print(aspirin)

## Compute Maximal Common Substructure

In [ ]:
from chembl_webresource_client.utils import utils

smiles = ["O=C(NCc1cc(OC)c(O)cc1)CCCC/C=C/C(C)C",
          "CC(C)CCCCCC(=O)NCC1=CC(=C(C=C1)O)OC", "c1(C=O)cc(OC)c(O)cc1"]
mols = [utils.smiles2ctab(smile) for smile in smiles]
sdf = ''.join(mols)
result = utils.mcs(sdf)
display(result)

## Compute various molecular descriptors

In [ ]:
from chembl_webresource_client.utils import utils
import json

aspirin = utils.smiles2ctab('O=C(Oc1ccccc1C(=O)O)C')
descs = json.loads(utils.chemblDescriptors(aspirin))[0]
display(descs)

## Compute structural alerts

In [ ]:
from chembl_webresource_client.utils import utils

mol = utils.smiles2ctab("O=C(Oc1ccccc1C(=O)O)C")
alerts = json.loads(utils.structuralAlerts(mol))
for a in alerts[0]:
    display(a)
    
pd.DataFrame(alerts[0])

## Standardize a molecule

In [ ]:
from chembl_webresource_client.utils import utils
mol = utils.smiles2ctab("[Na]OC(=O)Cc1ccc(C[NH3+])cc1.c1nnn[n-]1.O")
st = json.loads(utils.standardize(mol))
print(st[0]['standard_molblock'])

## Calculate the parent molecule

In [ ]:
from chembl_webresource_client.utils import utils

mol = utils.smiles2ctab("[Na]OC(=O)Cc1ccc(C[NH3+])cc1.c1nnn[n-]1.[Na]")
par = json.loads(utils.getParent(mol))
print(par[0]['parent_molblock'])